In [2]:
pip install cvxpy

     |████████████████████████████████| 1.1 MB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 456 kB 20.8 MB/s eta 0:00:01
     |████████████████████████████████| 254 kB 27.1 MB/s eta 0:00:01
     |████████████████████████████████| 234 kB 19.7 MB/s eta 0:00:01
     |████████████████████████████████| 108 kB 20.2 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 108 kB 19.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression as LR
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from scipy import stats
from scipy.special import binom
import matplotlib.pyplot as plt
import time 
import cvxpy as cp
import abess
from abess.linear import LinearRegression
from scipy.linalg import toeplitz
import gurobipy as gp

In [4]:
'''
S <- stored as p-dim np array of 0,1 indicating which variables active

'''



def compare_to_rows(SS, S):
    # S <- p-dim array, SS <- (m,p) array
    # returns -1 if not present otherwise row number
    logic = np.any(np.all(SS == S, axis =1))
    if logic == True:
        return np.where(logic == True)[0][0]
    else:
        return -1


class ebreg:

    def __init__(self, tuning_parameters):

        #tuning parameters <- dictionary
        self.s = tuning_parameters['s']
        self.B = tuning_parameters['B']
        self.epsilon = tuning_parameters['epsilon']
        self.sensitivity_scaling = tuning_parameters['sensitivity_scaling']

        # MCMC parameters
        self.max_iter = tuning_parameters['max_iter']
        
        # some options
        self.standardize = tuning_parameters["standardization"]
        self.initialization = tuning_parameters["initialization"]

    def fit(self, X, y):
        self.n, self.p = X.shape
        #if self.n >= 100: self.burn_in = 500
        # standardize X
        if self.standardize is True:
            scaler1 = StandardScaler(with_mean=True, with_std=False).fit(X)
            X = scaler1.transform(X)

            y = y - y.mean()

        self.X = X
        self.y = y
        

        self.MCMC()

   

    def draw_q(self, S):
        S1 = S.copy()
        s = S.sum()
        idx_0 = np.where(S1 == 0)[0]
        idx_1 = np.where(S1 == 1)[0]
        S1[np.random.choice(idx_1, 1)] = 0
        S1[np.random.choice(idx_0, 1)] = 1

        return S1

    def OLS_pred_and_pi_n(self, S):
        X_S = self.X[:, S == 1]
        y = self.y

        reg = LR(fit_intercept=False).fit(X_S, y)  # what happens when singular
        Y_S = reg.predict(X_S)

        epsilon = self.epsilon
        Du = self.sensitivity_scaling #2 * self.ymax**2 + 2 * self.s * (self.xmax**2) *  self.sensitivity_scaling * sigma2
        # s = S.sum()
        log_pi_n =  - epsilon * np.linalg.norm(y - Y_S)**2/(Du)  # np.log(gamma + alpha/sigma2)
        return Y_S, log_pi_n
                    
    def regOLS_pred_and_pi_n(self, S):
        X_S = self.X[:, S == 1]
        y = self.y
        
        b = cp.Variable(shape = X_S.shape[1])
        constraints = [cp.norm1(b) <= B]
        
        obj = cp.Minimize(cp.sum_squares(X_S@b - y))

        # Form and solve problem.
        prob = cp.Problem(obj, constraints)
        prob.solve() 
        Y_S = X_S @ b.value
        Du = self.sensitivity_scaling #2 * self.ymax**2 + 2 * self.s * (self.xmax**2) *  self.sensitivity_scaling * sigma2
        # s = S.sum()
        log_pi_n =  - self.epsilon * np.linalg.norm(y - Y_S)**2/(Du)
        
        return Y_S, log_pi_n
    
    
    def initialize(self):
        
        S = np.zeros(self.p)
        
        
        if self.initialization == "Lasso":
            reg = LassoCV(n_alphas=100, fit_intercept=False,
                          cv=5, max_iter=2000).fit(self.X, self.y)
            scores = np.abs(reg.coef_)
            s_max_scores_id = np.argsort(scores)[::-1][:s]
            S[s_max_scores_id] = 1
        
        if self.initialization == "MS":
            c = X.T@y/self.n
            c1 = np.abs(c)
            c2 = np.argsort(c1)[::-1][:s]
            S[c2] = 1
        
        else: S[np.random.choice(self.p, self.s, replace= False)] = 1
        
        self.initial_state = S
        
        self.S = S
        
        
        return S

    def MCMC(self):
        max_iter = self.max_iter
        
        # initialize
        S = self.initialize()
        self.S_list = [self.S]
        Y_S, log_pi_n = self.regOLS_pred_and_pi_n(self.S)
        self.Y_S_old = Y_S
        self.log_pi_n_list = [log_pi_n]
        self.RSS = np.array([np.linalg.norm(self.y - Y_S)**2/np.linalg.norm(self.y)**2])
        #self.F1 = [0]

        

        S = self.S
        y = self.y

        iter1 = 0
        no_acceptances = 0

        while (iter1 < max_iter):

            # proposal draw
            S_new = self.draw_q(S)
            Y_S_new, log_pi_n_new = self.regOLS_pred_and_pi_n(S_new)
            

            # compute hastings ratio
            try: HR = np.exp(log_pi_n_new - log_pi_n)
            except ValueError: print('Hastings ratio uncomputable')
            R = np.min([1, HR])
            if stats.uniform.rvs() <= R:
                # accept
                self.RSS = np.vstack((self.RSS, np.linalg.norm(self.y - Y_S_new)**2/np.linalg.norm(self.y)**2))
                self.S_list.pop()
                self.S_list.append(S_new)
                self.Y_S_old = Y_S_new
                S = S_new
                log_pi_n = log_pi_n_new
                no_acceptances += 1
            else:
                self.RSS = np.vstack((self.RSS, np.linalg.norm(self.y - self.Y_S_old)**2/np.linalg.norm(self.y)**2))

        

            iter1 += 1

        
        self.acceptance = no_acceptances 


In [7]:
# Example
np.random.seed(1)

data_type = 'Uniform' # Uniform or ar1
n = 900
p = 2000
s = 4
B = 3.5
rho = 0.3
signal = 'weak'
#Sigma = np.eye(p)*(1-rho) + rho * np.ones((p, p))

if data_type == 'Uniform':
    X = np.random.uniform(-1, 1, n*p).reshape(n,p)
    e = np.random.uniform(-0.1, 0.1, size = n)
if data_type == 'ar1':  
    Sigma = toeplitz([rho**i for i in range(p)])
    X = np.random.multivariate_normal(mean = np.zeros(p), cov = Sigma, size = n)
    #X = X/np.max(np.abs(X))
    scaler1 = StandardScaler(with_mean=True, with_std= True).fit(X)
    X = normalize(scaler1.transform(X), axis=0)
    e = np.random.uniform(-.1, .1,n)
beta = np.zeros(p)

if signal == 'weak': beta[0:s] = 2.0 * np.sqrt(1 * np.log(p)/n)
else: beta[0:s] = 2.0 * np.sqrt(s * np.log(p)/n)

#scaler1 = StandardScaler(with_mean=True, with_std=False).fit(X)
#X = scaler1.transform(X)
#X = X

y = X @ beta + e
y1 = y
X1 = X

In [8]:

Du = (B * 1 + np.abs(y1).max())**2
print(Du)
np.abs(y).max(), beta.sum()

17.399388331296915


(0.6712574041045363, 0.7351929130134585)

In [9]:
eps_list = [0.5, 1, 2.5, 3, 5, 10]
for eps in eps_list:
    tuning_parameters = {'epsilon': eps,
                         'sensitivity_scaling': Du,
                         's'      : s,
                         'B'      : B,   
                         'max_iter': 100000,
                        'initialization': 'Random',
                        'standardization': False}

    model = ebreg(tuning_parameters)

    #y = y/np.abs(y).max()

    def fit_MCMC(i):
        model.fit(X1,y1)
        #print(str(i)+'th chain fitting complete')
        S_hat = np.where(model.S_list[-1]>0)[0]
        S = np.where(beta!=0)[0]
        prec = len(np.intersect1d(S_hat, S))/max(1,len(S_hat))
        recall = len(np.intersect1d(S_hat, S))/len(S)
        F1 = 0
        if prec + recall>0: F1 = 2 * (prec * recall)/(prec + recall)
        return {'RSS': model.RSS, 'F1': F1}


    num_chains = 10 
    from joblib import Parallel, delayed

    start = time.time()
    results = Parallel(n_jobs= num_chains)(delayed(fit_MCMC)(i) for i in range(num_chains))
    end = time.time()
    print(end-start)
    model_abess = LinearRegression(support_size = s)
    model_abess.fit(X1, y1)
    S_abess = np.where(model_abess.coef_!=0)[0]
    S = np.where(beta!=0)[0]
    prec = len(np.intersect1d(S_abess, S))/max(1,len(S_abess))
    recall = len(np.intersect1d(S_abess, S))/len(S)
    F1_abess = 2/(1/prec + 1/recall)

    RSS_abess = np.linalg.norm(y1 - X1@model_abess.coef_)**2/np.linalg.norm(y1)**2
    RSS_true = np.linalg.norm(y1 - X1@beta)**2/np.linalg.norm(y1)**2

    plt.axhline(y = 1- RSS_true, color = 'b', linestyle = '--', label = '$R_{\gamma^*}$')
    plt.axhline(y = 1- RSS_abess, color = 'r', linestyle = '--', label = '$R_{\gamma_{BSS}}(non-private)$')

    for i in range(num_chains):
        RSS = results[i]['RSS']
        if i==0: plt.plot(range(len(RSS)), (1-RSS), color = 'grey', alpha = 0.3, label = '$R_{\gamma_{BSS}}(\epsilon = $' + 
                          str(tuning_parameters['epsilon']) + ')')
        else: plt.plot(range(len(RSS)), (1-RSS), color = 'grey', alpha = 0.3)
    
    F1 = np.mean([results[i]['F1'] for i in range(num_chains)])
    plt.text(1, 0.8, 'Mean F1  = ' + str(F1) + '\n K = ' + str(B), size = 13, bbox = {'fc': 'wheat', 'alpha':0.8})
    #plt.ylim(ymax= 1)
    plt.xlabel('iterations', size = 13)
    #plt.xlim(xmax = 1e5)
    plt.ylabel('$R_\gamma$', size = 13)
    #plt.title()
    #plt.yscale('log')  
    plt.xscale('log')
    plt.legend(loc = 'center left',fontsize = 13)
    #plt.show()
    name = str(data_type)+'_design_multiple_MH_chain_'+str(num_chains)+'_epsilon_'+str(tuning_parameters['epsilon'])+'_B_'+ str(B)+ '_s_'+str(s)+'_p_'+str(p)+'_n_'+str(n)+'.pdf'

    if signal == 'weak': plt.savefig('weak_'+name)
    else: plt.savefig(name)
    plt.clf()

1012.7551815509796
1010.2512736320496
1015.9584033489227
1012.9063003063202
1035.537309885025
1046.2966017723083


<Figure size 640x480 with 0 Axes>

In [10]:
F1 = np.mean([results[i]['F1'] for i in range(num_chains)])
F1_dict = {'MH_mean': F1, 'BSS': F1_abess}

# if signal == 'weak': np.save('weak_F1'+name, F1_dict, allow_pickle = True)
# else: np.save('F1'+name, F1_dict, allow_pickle = True)
F1_dict, print(end - start)

1046.2966017723083


({'MH_mean': 0.275, 'BSS': 1.0}, None)

In [19]:
RSS_true = np.linalg.norm(y1 - X1@beta * np.abs(X).max()/np.abs(y).max())**2/np.linalg.norm(y1)**2
RSS_true

0.23086597382090704

In [22]:
sum(beta)

1.688816450596388

In [28]:
model.fit(X,y)

In [27]:
model.S_list[0][:10]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [49]:
1-RSS_true, 1- RSS_abess

(0.24965812470283621, 0.25109691429141046)

In [122]:
n = 'F1'+name+'.npy'
np.load('F1Uniform_design_multiple_MH_chain_10_epsilon_10_s_10_p_2000_n_900.pdf.npy', allow_pickle = True)

array({'MH_mean': 0.9, 'BSS': 1.0}, dtype=object)

2th chain fitting complete


ZeroDivisionError: float division by zero

In [79]:

X_S = X1[:,np.random.choice(p, s)] # np.random.choice(p, s)
y = y1

b = cp.Variable(shape = X_S.shape[1])
constraints = [cp.norm1(b) <= B]

obj = cp.Minimize(cp.sum_squares(X_S@b - y))

# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve() 


161.63479946902982

In [59]:
prob.value, B


(142.06341132021524, 2)

In [61]:
np.linalg.norm(X_S@b.value - y)**2

142.06341132021524

In [53]:
sum(abs(b.value))

0.0878052498990281

In [40]:
np.random.choice(6, 3)

array([5, 0, 2])

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 37.9 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.
